<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#установка библиотек в тихом (quiet) режиме, когда логи не выводятся
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 6.9 MB/s 
     |████████████████████████████████| 7.6 MB 34.7 MB/s 
     |████████████████████████████████| 163 kB 75.0 MB/s 


In [2]:
!pip install -q torch

In [3]:
! pip install -q wordcloud
from wordcloud import WordCloud, STOPWORDS

## Подготовка

In [4]:
import numpy as np
import pandas as pd
import torch
import re
import transformers

import nltk
import lightgbm as lgb
#загрузим список стоп-слов
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tqdm import notebook
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
#from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [5]:
# читаем данные из .csv файла с помощью метода read_csv()
try:
    df = pd.read_csv('D:\\python\\project_13\\toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

# с помощью метода info() изучим структуру таблицы: типы данных, количество строк, столбцов, пропущенных данных.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

In [7]:
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


- В таблице 159292 столбцов.
- Пропуски не обнаружены.
- Имеются три столбца, основная информация хранится в столбце text
- Целевой признак хранится в столбце toxic, который содержит в себе булево выражение, отвечая на вопрос, токсичный ли комментарий или нет.
- Столбец Unnamed: 0 по всей видимости является дублирующим индексы таблицы. Посчитаем, что далее он нам не понадобится.
- Понадобится предобработка данных в столбце text, а именно:
        - токенизация, разбиение на отдельные фразы, например знаки пунктуации будут лишними при обучении моделей
        - лемматизация (для лучшего обучения моделей необходимо далее привести выражения к леммам, то есть к изначальным формам слов)
        - также необходимо избавиться от слов, которые не несут в себе смысловой нагрузки


- посчитаем соотношение токсичных комментариев к остальным.

In [8]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

10,16 %  от общего количества данных токсичны, если рассматривать грубо, то каждый десятый комментарий.

In [10]:
df.duplicated().sum()

0

Произведена проверка на дубликаты. Последние не обнаружены.

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,159292.0,79725.697242,46028.837471,0.0,39872.75,79721.5,119573.25,159450.0
toxic,159292.0,0.101612,0.302139,0.0,0.00,0.0,0.00,1.0


Удалим лишний столбец.

In [12]:
df = df.drop('Unnamed: 0', axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Сделаем выборку df_sample для обучения модели DistilBert.

In [14]:
df_sample = df.sample((10000), random_state=123).reset_index(drop=True) 
df_sample['toxic'].value_counts()

0    9011
1     989
Name: toxic, dtype: int64

Соотношение негативных к позитивным комментариям сохранилось. Каждый десятый комментарий негативный.

## Обучение

https://habr.com/ru/post/498144/

https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

Обучим переменную модель model, которая содержит предварительно обученную модель Дистилберта DistilBertModel - версию BERT, которая меньше, но намного быстрее и требует гораздо меньше памяти.

https://habr.com/ru/post/436878/

дополнительно для понимания работы модели BERT изучено

https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

In [15]:
model_class = transformers.AutoModel
tokenizer_class = transformers.AutoTokenizer
pretrained_weights = ('unitary/toxic-bert')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Некоторые веса контрольной точки модели в unitary/toxic-bert не использовались при инициализации модели Bert: ['classifier.base', 'classifier.вес']
- - Это ожидается, если вы инициализируете модель Bert из контрольной точки модели, обученной для другой задачи или с другой архитектурой (например, инициализируете модель BertForSequenceClassification из модели BertForPreTraining).
- - Это НЕ ожидается, если вы инициализируете модель Bert с контрольной точки модели, которая, как вы ожидаете, будет точно идентичной (инициализация модели BertForSequenceClassification из модели BertForSequenceClassification).

## Модель №1: Подготовка набора данных

Прежде чем мы сможем раздавать предложения Берту, нам нужно выполнить некоторую минимальную обработку, чтобы привести их в требуемый формат.

## Токенизация

Наш первый шаг - обозначить предложения - разбить их на слова и подслова в удобном для Берта формате.
- сразу обрезаем слишком длинные последовательности до 512 токенов

In [16]:
tokenized = df_sample['text'].apply((lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True)))

## Паддинг (отступ)

- После токенизации tokenized представляет собой список предложений - каждое предложение представлено в виде списка токенов. 
- Мы хотим, чтобы БЕРТ обрабатывал наши примеры все сразу (как один пакет). Просто так будет быстрее. 
- По этой причине нам нужно заполнить все списки одинаковым размером, чтобы мы могли представлять входные данные в виде одного двумерного массива, а не списка списков (разной длины).

In [17]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Наш набор данных теперь находится в дополненной переменной, мы можем просмотреть его размеры ниже:

In [18]:
np.array(padded).shape

(10000, 512)

## Маскировка

- Если мы напрямую отправим его Берту, это немного запутает ситуацию. 
- Нам нужно создать другую переменную, чтобы указать ей игнорировать (маскировать) добавленные нами отступы, когда она обрабатывает свои входные данные. Вот что такое attention_mask:

In [19]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 512)

## Модель №1: А теперь обучение!

Теперь, когда у нас есть наша модель и входные данные, давайте запустим нашу модель!

Функция model() прогоняет наши предложения через BERT. Результаты обработки будут возвращены в последние _ скрытые _ состояния last_hidden_states.

Давайте нарежем только ту часть выходных данных, которая нам нужна. Это выходные данные, соответствующие первому символу каждого предложения. Способ, которым BERT выполняет классификацию предложений, заключается в том, что он добавляет маркер с именем [CLS] (для классификации) в начале каждого предложения. Выходные данные, соответствующие этому токену, можно рассматривать как вложение для всего предложения.

Мы сохраним их в переменной features, поскольку они будут служить функциями для нашей модели логистической регрессии.

In [20]:
batch_size = 100
embeddings = []

from tqdm import tqdm

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

for i in tqdm(range(tokenized.values.shape[0] // batch_size)):
    # преобразуем батч с токенизированными твитами в тензор 
    # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
    input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    # создаем тензор и для подготовленной маски
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
    
    # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
    # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
    with torch.no_grad():
        model.to(device)
        last_hidden_states = model(input_ids, attention_mask = attention_mask_batch)
    
    # в итоге собираем все эмбеддинги твитов в features
    embeddings.append(last_hidden_states[0][:,0,:].cpu().numpy())
    
    torch.cuda.empty_cache()

cuda:0


100%|██████████| 100/100 [05:50<00:00,  3.50s/it]


In [21]:
features = np.concatenate(embeddings)
target = df_sample['toxic']

In [22]:
features.shape

(10000, 768)

In [23]:
features_train, features_test, target_train, target_test = train_test_split(features, target, stratify=target,  test_size=0.1, random_state=123)

In [24]:
features_train.shape

(9000, 768)

Обучим модель LogisticRegression

In [25]:
model = LogisticRegression(random_state=123)
model.fit(features_train, target_train)
pred = model.predict(features_test)
print('f1_score на тестовой выборке для LogisticRegression: ', round(f1_score(target_test, pred), 3))

f1_score на тестовой выборке для LogisticRegression:  0.929


Проверка DummyClassifier

In [26]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='stratified')

dummy.fit(features_train, target_train)

predictions_dummy = dummy.predict(features_test)

print('f1_score DummyClassifier на тестовой выборке:', round(f1_score(target_test, predictions_dummy), 3))

f1_score DummyClassifier на тестовой выборке: 0.072


In [27]:
from lightgbm import LGBMRegressor

model_lgbmr = LGBMRegressor()

parameters = {'boosting_type': 'gbdt', 'random_state': 123 }

model_lgbmr = LGBMRegressor(**parameters)

params_lgbmr = [{'class_weight':['balanced', None],
                 'C':[1,10,100]
                       }]

grid_search_lgbmr = GridSearchCV(model_lgbmr, 
                        params_lgbmr, 
                        cv = 5, 
                        verbose = 0, 
                        n_jobs=-1,
                        scoring='neg_root_mean_squared_error')

grid_search_lgbmr.fit(features_train, target_train)

best_index = grid_search_lgbmr.best_index_

fit_time_grid = grid_search_lgbmr.cv_results_['mean_fit_time'][best_index]
score_time_grid = grid_search_lgbmr.cv_results_['mean_score_time'][best_index]
test_score_grid = abs(grid_search_lgbmr.cv_results_['mean_test_score'][best_index])

print('Время обучения: {},  время предсказания: {} и качество модели: {}'.format(fit_time_grid.round(3), score_time_grid.round(3), test_score_grid.round(1)))

Время обучения: 23.49,  время предсказания: 0.028 и качество модели: 0.1


In [28]:
model = RandomForestClassifier(random_state=123, class_weight = 'balanced') 
model.fit(features_train, target_train)
predictions = model.predict(features_test)

f1 = f1_score(target_test, predictions)
print('RandomForestClassifier', f1)

RandomForestClassifier 0.923076923076923


RandomForestClassifier показывает результат 0.923

In [29]:
%%time
results_rfc = []

for depth in range(1,11):
    
    for estimator in range(10, 51, 10):
        
        model = RandomForestClassifier(random_state=123, 
                                       class_weight = 'balanced', 
                                       n_estimators=estimator, 
                                       max_depth=depth) 
        
        model.fit(features_train, target_train)
        predictions = model.predict(features_test)

        f1 = f1_score(target_test, predictions)
        results_rfc.append({'Model': 'RandomForestClassifier', 
                            'Hyperparameters': {'random_state': 123, 
                                                'class_weight': 'balanced',
                                                'n_estimators': estimator, 
                                                'max_depth':depth}, 
                            'F1 score': f1})

CPU times: user 1min 51s, sys: 109 ms, total: 1min 51s
Wall time: 1min 51s


In [30]:
results_rfc

[{'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 10,
   'max_depth': 1},
  'F1 score': 0.8255319148936171},
 {'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 20,
   'max_depth': 1},
  'F1 score': 0.8533333333333334},
 {'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 30,
   'max_depth': 1},
  'F1 score': 0.8648648648648648},
 {'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 40,
   'max_depth': 1},
  'F1 score': 0.8533333333333334},
 {'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 50,
   'max_depth': 1},
  'F1 score': 0.8495575221238938},
 {'Model': 'RandomForestClassifier',
  'Hyperparameters

С поиском гиперпараметров удалось добиться результата F1 0.93{'Model': 'RandomForestClassifier',
  'Hyperparameters': {'random_state': 123,
   'class_weight': 'balanced',
   'n_estimators': 50,
   'max_depth': 10},
  'F1 score': 0.9300000000000002}]

### Дополнительное альтернативное решение

Попробуем идти другим путем и выполним предобработку текста

Очистим текст от лишних символов и лемматизируем

Далее функция очистки текста

In [31]:
def clear_text(text):
    
    text = text.lower()
    
    text = re.sub(r'[^a-zA-Z ]', ' ', text).strip()
    
    # Удаляем html теги
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # Удаляем единичные символы
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Удаляем единичные символы из начала строки
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    
    # Заменяем несколько пробелов на один
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Удаляем 'b'
    text = re.sub(r'^b\s+', '', text)
        
    return text

In [32]:
df['text'] = df['text'].apply(clear_text)
display(df.head())

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour m seem...,0
2,hey man m really not trying to edit war it jus...,0
3,more can make any real suggestions on improvem...,0
4,you sir are my hero any chance you remember wh...,0


Сохранение данных для пропуска шага лемматизации

Вновь разделим данные на обучающую и тестовую выборки в соотношении 80:20.

In [35]:
features = df['text']
target = df['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=123)

Создадим признаки с учетом стоп-слов и выделим целевые признаки

In [36]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(features_train)

features_train = tf_idf.transform(features_train)
features_test = tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
model = LogisticRegression(random_state=123)
model.fit(features_train, target_train)
pred = model.predict(features_test)
print('f1_score на тестовой выборке для LogisticRegression без подбора параметров: ', round(f1_score(target_test, pred), 3))

f1_score на тестовой выборке для LogisticRegression без подбора параметров:  0.739


Результат уже лучше(0.739), но тем не менее не достигнуто пороговое значение f1_score 0,75
Попробуем подобрать гиперпараметры для модели LogisticRegression

In [38]:
def model_look (model, parameters, features, target):
    model_grid = GridSearchCV(model, parameters, cv=5, scoring='f1', verbose=5)
    model_grid.fit(features, target)
    return model_grid.best_score_, model_grid.best_params_

In [39]:
parameters = {'C': range(23, 26, 1), 'max_iter': range(8, 10, 1)}
f1, best_parameters = model_look (LogisticRegression(random_state=12345, solver='liblinear'), parameters, features_train, target_train)
display('Лучшие параметры модели линейной регрессии:', best_parameters)
display('Дали f1 модели:', f1)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ..................C=23, max_iter=8;, score=0.777 total time=   0.8s
[CV 2/5] END ..................C=23, max_iter=8;, score=0.751 total time=   0.8s
[CV 3/5] END ..................C=23, max_iter=8;, score=0.762 total time=   0.9s
[CV 4/5] END ..................C=23, max_iter=8;, score=0.770 total time=   0.9s
[CV 5/5] END ..................C=23, max_iter=8;, score=0.770 total time=   0.9s
[CV 1/5] END ..................C=23, max_iter=9;, score=0.779 total time=   1.0s
[CV 2/5] END ..................C=23, max_iter=9;, score=0.749 total time=   0.9s
[CV 3/5] END ..................C=23, max_iter=9;, score=0.760 total time=   0.9s
[CV 4/5] END ..................C=23, max_iter=9;, score=0.767 total time=   1.5s
[CV 5/5] END ..................C=23, max_iter=9;, score=0.772 total time=   1.5s
[CV 1/5] END ..................C=24, max_iter=8;, score=0.778 total time=   1.4s
[CV 2/5] END ..................C=24, max_iter=8;,

'Лучшие параметры модели линейной регрессии:'

{'C': 25, 'max_iter': 8}

'Дали f1 модели:'

0.7660256656410953

Неплохой результат f1 модели 0.76, пройдено пороговое значение, но применение модели Берт дало существенно более высокие результаты.

## Выводы

- Данные успешно загружены и обработаны. 
- Лучший результат, которого удалось добиться - f1_score на тестовой выборке для LogisticRegression:  0.929
- На Обучение модели было затрачено 05:38 и f1_score выше требуемой 0,75 существенно.
- при поиске гиперпараметров для Случайного Леса качество метрики f1 не увеличилось значительно и значение было всего лишь на 0.001 выше.
- Применение модели берт дает более высокий результат итоговых метрик

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны